#### importamos las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)

#### LIMPIEZA DE TEXTO

In [3]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
# Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
# Se eliminan los espacios de sobra
# Se eliminan \r, \t, \v, \f, \a
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)
pprint(df1[0])


('Informa a continuacion: \n'
 '\n'
 'J.T. 13 \n'
 'PERITO MEDICO PRESENTA INFORME PERICIAL \n'
 ' \n'
 'Autos: “BOIERO FRANCO IVAN c/SWISS MEDICAL ART S.A. s/Accidente - Ley '
 'Especial” \n'
 'Expediente N.º: 71399/2017 \n'
 ' \n'
 'I-PROEMIO: \n'
 ' \n'
 'Sr. JUEZ NACIONAL: \n'
 ' \n'
 'HORTAS MARIA ANDREA S. perito medico de oficio en los siguientes \n'
 'autos, con domicilio constituido en la calle Austria 1754 3º piso Dpto. 10 '
 'de esta \n'
 'Capital Z 172, tel. 4325-8802 domicilio electronico mashortas@hotmail.com, '
 '27-\n'
 '16523394-3 en cumplimiento lo dispuesto por V.S. referente a los presentes '
 'autos \n'
 'y despues de haber examinado a quien dijo ser Boiero Franco Ivan quien se \n'
 'tomaron en el momento del examen el estado clinico actual se consideraron '
 'la \n'
 'capacidad generica y en particular los puntos de pericia solicitados por las '
 'partes \n'
 'y/o V.S.: \n'
 ' \n'
 ' \n'
 ' \n'
 'II-ANTECEDENTES DE INTERES MEDICO-LEGAL: \n'
 'Foja 3 Inicia la dema

In [27]:
#Segunda fase de limpieza
import re

def limpiarTexto2(text):
    text = re.sub(' +','',text)
    text = re.sub('\n ','',text)
    text = re.sub('\xcc\x81','',text)
    #text = re.sub(' +\n','\n',text)
    text = re.sub('\d{1,100}\n','',text)
    text = re.sub('\x0c','',text)
    #text = re.sub('^ +','',text)
    text = re.sub('n.º.','',text)
    text = re.sub('[a-zA-z-.]@[a-zA-Z].com','',text)
    #text = re.sub('\d+','',text)
    text = re.sub(' +\n+ ','\n',text)
    text = re.sub(' \\n\\n','\n',text)
    #text = text.strip().replace('\n','')
    return text

dfLimpio = df1.apply(limpiarTexto2)
pprint(dfLimpio[0])

(' \\nInforma a continuacio\\xcc\\x81n \\nJT \\nPERITO MEDICO PRESENTA INFORME '
 'PERICIAL \\n \\nAutos BOIERO FRANCO IVAN cSWISS MEDICAL ART SA sAccidente '
 'Ley Especial \\nExpediente No \\n \\nIPROEMIO \\n \\nSr JUEZ NACIONAL \\n '
 '\\nHORTAS MARIA ANDREA S perito me\\xcc\\x81dico de oficio en los siguientes '
 '\\nautos con domicilio constituido en la calle Austria piso Dpto de esta '
 '\\nCapital Z tel domicilio electro\\xcc\\x81nico mashortashotmailcom \\n en '
 'cumplimiento lo dispuesto por VS referente a los presentes autos \\ny '
 'despue\\xcc\\x81s de haber examinado a quie\\xcc\\x81n dijo ser Boiero '
 'Franco Iva\\xcc\\x81n quie\\xcc\\x81n se \\ntomaron en el momento del examen '
 'el estado cli\\xcc\\x81nico actual se consideraron la \\ncapacidad '
 'gene\\xcc\\x81rica y en particular los puntos de pericia solicitados por las '
 'partes \\nyo VS \\n \\n \\n \\nIIANTECEDENTES DE INTERES MEDICOLEGAL \\nFoja '
 'Inicia la demanda \\nFoja vta Hechos accidente in itinere re

#### Eliminacion de stopwords y lematizacion.

In [11]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

df1 = df1.apply(remove_stops)

#print(df1)

In [12]:
#para ver un expediente, verifico que me tokenize correctamente.
from pprint import pprint
pprint(df1[0])

['informa',
 'continuacion',
 'j',
 't',
 'perito',
 'medico',
 'presenta',
 'informe',
 'pericial',
 'autos',
 'boiero',
 'franco',
 'ivan',
 'c',
 'swiss',
 'medical',
 'art',
 's',
 's',
 'accidente',
 'ley',
 'especial',
 'expediente',
 'n',
 'i',
 'proemio',
 'sr',
 'juez',
 'nacional',
 'hortas',
 'maria',
 'andrea',
 's',
 'perito',
 'medico',
 'oficio',
 'siguientes',
 'autos',
 'domicilio',
 'constituido',
 'calle',
 'austria',
 'piso',
 'dpto',
 'capital',
 'z',
 'tel',
 'domicilio',
 'electronico',
 'mashortas',
 'hotmail',
 'com',
 'cumplimiento',
 'dispuesto',
 'v',
 's',
 'referente',
 'presentes',
 'autos',
 'despues',
 'haber',
 'examinado',
 'dijo',
 'ser',
 'boiero',
 'franco',
 'ivan',
 'tomaron',
 'momento',
 'examen',
 'clinico',
 'actual',
 'consideraron',
 'capacidad',
 'generica',
 'particular',
 'puntos',
 'pericia',
 'solicitados',
 'partes',
 'v',
 's',
 'ii',
 'antecedentes',
 'interes',
 'medico',
 'legal',
 'foja',
 'inicia',
 'demanda',
 'foja',
 'vta',
 

In [1]:
from sklearn.feature_extraction.text import CountVectorizer 

documento = df1.apply(str)  
# crear la transformación  
vectorizer = CountVectorizer() 
# tokenizar y construir el vocabulario
vectorizer.fit(documento) 
  
print("Vocabulary: ", vectorizer.vocabulary_) 
# codificador de documentos
vector = vectorizer.transform(documento) 

# resumir vector codificado
#print("El documento codificado es:") 
#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

NameError: name 'df1' is not defined

In [81]:
def general(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt

pprint(general(df[0]))

('Informa a continuacion: \n'
 '\n'
 'J.T. 13 \n'
 'PERITO MEDICO PRESENTA INFORME PERICIAL \n'
 ' \n'
 'Autos: “BOIERO FRANCO IVAN c/SWISS MEDICAL ART S.A. s/Accidente - Ley '
 'Especial” \n'
 'Expediente N.º: 71399/2017 \n'
 ' \n'
 'I-PROEMIO: \n'
 ' \n'
 'Sr. JUEZ NACIONAL: \n'
 ' \n'
 'HORTAS MARIA ANDREA S. perito medico de oficio en los siguientes \n'
 'autos, con domicilio constituido en la calle Austria 1754 3º piso Dpto. 10 '
 'de esta \n'
 'Capital Z 172, tel. 4325-8802 domicilio electronico mashortas@hotmail.com, '
 '27-\n'
 '16523394-3 en cumplimiento lo dispuesto por V.S. referente a los presentes '
 'autos \n'
 'y despues de haber examinado a quien dijo ser Boiero Franco Ivan quien se \n'
 'tomaron en el momento del examen el estado clinico actual se consideraron '
 'la \n'
 'capacidad generica y en particular los puntos de pericia solicitados por las '
 'partes \n'
 'y/o V.S.: \n'
 ' \n'
 ' \n'
 ' \n'
 'II-ANTECEDENTES DE INTERES MEDICO-LEGAL: \n'
 'Foja 3 Inicia la dema

In [127]:
def limpiarTextoPrueba(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

#pprint(df[1])
pprint(limpiarTextoPrueba(df[1]))

('\n'
 'JUZG. 4PERITO MEDICO PRESENTA INFORME MÉDICO-PERICIAL\n'
 'Al JUZGADO NACIONAL DE PRIMERA INSTANCIA DEL TRABAJO  40, sito en\n'
 'Diag. Roque Sáenz Peña , piso 5º de CABA\n'
 'SEÑOR JUEZ NACIONAL\n'
 'I)\n'
 'PROEMIO\n'
 'El que suscribe Dr. RICCO, ITALO ROBERTO, M.N. 39., domicilio electrónico\n'
 ', especialista de la U.B.A. en Medicina del Trabajo  y Enfermedades\n'
 'Infecciosas y de la A.M.A. en Clínica Médica, designado perito médico de '
 'oficio en los\n'
 'presentes autos caratulados: “GONZÁLEZ, FLAVIA MELINA c/ PROVINCIA ART\n'
 'S.A. s/ ACCIDENTE – LEY ESPECIAL – CAUSA /, se dirige\n'
 'respetuosamente a V.S. para informar que, después de haber examinado a la '
 'actora,\n'
 'identificándose con D.N.I.  26.., de quien se tomaron en el momento del\n'
 'examen la fotografía de identificación que se adjunta, su estado clínico '
 'actual, los\n'
 'estudios complementarios solicitados, su capacidad y en particular, los '
 'puntos de\n'
 'pericia solicitados por las partes